In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Read and Write Parquet to GCS") \
    .getOrCreate()

spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile","dtc-de-zoomcamp-410523-29fcc5b444c6.json")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/05 02:37:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/05 02:37:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Specify the path to the Parquet files in GCS
bucket_name="mage-zoomcamp-jonah-oliver"
path=f"gs://{bucket_name}/ny_taxi_data/service=green/year=2019/*"

df=spark.read.parquet(path)

In [51]:
# Show the DataFrame
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2018-12-21 15:17:29|  2018-12-21 15:18:57|                 N|         1|         264|         264|              5|          0.0|        3.0|  0.5|    0.

In [26]:
df.registerTempTable('green')


In [18]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2019-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY 
    1, 2
""")

In [28]:
path=f"gs://{bucket_name}/ny_taxi_data/service=yellow/year=2019/*"

df=spark.read.parquet(path)

In [30]:
df.registerTempTable('yellow')

In [31]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [32]:
df_yellow_revenue.show()

+-------------------+----+------+--------------+
|               hour|zone|amount|number_records|
+-------------------+----+------+--------------+
|2088-01-24 00:00:00|  24|  15.3|             1|
|2088-01-24 00:00:00|  41|   5.3|             1|
|2041-03-25 01:00:00| 151|  15.8|             1|
|2033-04-27 12:00:00| 107|   7.3|             1|
|2033-04-27 13:00:00| 263|   8.3|             1|
|2033-04-27 12:00:00| 229|  10.8|             1|
|2020-01-03 03:00:00|  97|  9.75|             1|
|2020-01-02 04:00:00| 100| 33.96|             1|
|2020-02-28 17:00:00| 237| 21.36|             1|
|2020-01-15 17:00:00| 233| 11.33|             1|
|2020-01-24 07:00:00| 164| 23.16|             1|
|2020-01-02 00:00:00| 163| 17.25|             1|
|2020-02-07 09:00:00| 233| 32.76|             1|
|2020-01-15 17:00:00| 263| 11.84|             1|
|2020-02-06 01:00:00| 263|  34.3|             1|
|2020-01-02 01:00:00| 164|   8.8|             1|
|2020-01-02 22:00:00| 158| 14.16|             1|
|2020-01-24 07:00:00

In [33]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2019-01-01 00:00:00|   3|             90.02|             2|
|2019-01-01 00:00:00|   7| 847.5499999999994|            68|
|2019-01-01 00:00:00|  11|              10.3|             1|
|2019-01-01 00:00:00|  14| 77.39999999999999|             3|
|2019-01-01 00:00:00|  16|50.599999999999994|             2|
|2019-01-01 00:00:00|  17|170.85000000000002|            10|
|2019-01-01 00:00:00|  18|             75.71|             3|
|2019-01-01 00:00:00|  20|             47.55|             1|
|2019-01-01 00:00:00|  24|              75.5|             6|
|2019-01-01 00:00:00|  25|1161.3399999999995|            48|
|2019-01-01 00:00:00|  26|              21.3|             1|
|2019-01-01 00:00:00|  28|             21.51|             1|
|2019-01-01 00:00:00|  29|              22.1|             2|
|2019-01-01 00:00:00|  3

In [45]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [48]:
bucket_name="mage-zoomcamp-jonah-oliver"
path=f"gs://{bucket_name}/ny_taxi_data/service=green/year=2019/report/revenue/"

df_green_revenue_tmp \
    .repartition(20) \
    .write.parquet(path, mode='overwrite')

In [35]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')